## Testing GPU configuration (CuDNN)

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
print('Number of GPUs:', len(tf.config.experimental.list_physical_devices('GPU')))

Number of GPUs: 0


In [3]:
tf.test.is_built_with_cuda()

False

In [4]:
tf.test.is_built_with_gpu_support()

False

## Importing necessary libraries

In [5]:
import keras
from tensorflow.keras.layers import Layer
import keras.backend as K
import tensorflow as tf

In [6]:
import cv2
import numpy as np
from keras.utils import np_utils
from tensorflow.keras import initializers

In [7]:
import math
from tensorflow.keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

In [8]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D, Dropout, GlobalAveragePooling2D
from keras.layers.merge import concatenate

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense 
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping

## Loading the dataset

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
  train_datagen_with_aug = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   shear_range=0.2,
                                   zoom_range=[0.2,0.5],
                                   width_shift_range = 0.3,
                                   height_shift_range = 0.3,
                                   validation_split=0.2)

train_generator = train_datagen_with_aug.flow_from_directory(
    r'D:\My Work\Baranidharan Sir Research\cnn_xrays_images',
    target_size=(331, 331),
    batch_size=16,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
    ) # set as training data

validation_generator = train_datagen_with_aug.flow_from_directory(
    r'D:\My Work\Baranidharan Sir Research\cnn_xrays_images', # same directory as training data
    target_size=(331, 331),
    batch_size=16,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
    ) # set as validation data

input_img = Input(shape=(331, 331, 1))

Found 5121 images belonging to 3 classes.
Found 1278 images belonging to 3 classes.


## Constructing the Architecture

In [11]:
# kernel_init = keras.initializers.glorot_uniform()
# bias_init = keras.initializers.Constant(value=0.2)

In [12]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)

    pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

In [13]:
input_layer = Input(shape=(331, 331, 1))

# x = Conv2D(64, (7, 7), padding='valid', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = Conv2D(64, (7, 7), padding='valid', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(input_layer)
x = MaxPooling2D((3, 3), padding='valid', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='valid', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='valid', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPooling2D((3, 3), padding='valid', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPooling2D((3, 3), padding='valid', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='valid', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(3, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='valid', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(3, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPooling2D((3, 3), padding='valid', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(3, activation='softmax', name='output')(x)

In [14]:
model = Model(input_layer, [x, x1, x2], name='inception_v1')

In [15]:
model.summary()

Model: "inception_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 331, 331, 1) 0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 163, 163, 64) 3200        input_2[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 81, 81, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 81, 81, 64)   4160        max_pool_1_3x3/2[0][0]           
_______________________________________________________________________________________

In [16]:
epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(learning_rate=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

In [17]:
graph = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

graph

Epoch 1/25
321/321 [==============================] - 971s 3s/step - loss: 1.7216 - output_loss: 1.0760 - auxilliary_output_1_loss: 1.0747 - auxilliary_output_2_loss: 1.0771 - output_accuracy: 0.3915 - auxilliary_output_1_accuracy: 0.3958 - auxilliary_output_2_accuracy: 0.3823 - val_loss: 1.7177 - val_output_loss: 1.0740 - val_auxilliary_output_1_loss: 1.0725 - val_auxilliary_output_2_loss: 1.0730 - val_output_accuracy: 0.3858 - val_auxilliary_output_1_accuracy: 0.3858 - val_auxilliary_output_2_accuracy: 0.3858
Epoch 2/25
321/321 [==============================] - 1010s 3s/step - loss: 1.7154 - output_loss: 1.0731 - auxilliary_output_1_loss: 1.0683 - auxilliary_output_2_loss: 1.0725 - output_accuracy: 0.4015 - auxilliary_output_1_accuracy: 0.4103 - auxilliary_output_2_accuracy: 0.4007 - val_loss: 1.7002 - val_output_loss: 1.0652 - val_auxilliary_output_1_loss: 1.0541 - val_auxilliary_output_2_loss: 1.0628 - val_output_accuracy: 0.4859 - val_auxilliary_output_1_accuracy: 0.4100 - val_au

In [18]:
max_acc = max(graph.history['val_output_accuracy'])

print('The highest accuracy achieved using GoogleNet with input: 331x331 is',max_acc*100)

The highest accuracy achieved using GoogleNet with input: 331x331 is 48.591548204422
